**This jupyter notebook that demonstrates how to:**

1. Connect to Oracle database;
2. Run a query limiting sizes of results;
3. Create a csv file with results at google drive.

The CSV files creation process was necessary to make possible analysis and model tests without a database setup process.

In [0]:
#install 2 libs that are not found at google colab by default
!pip install google
!pip install cx_Oracle

In [0]:
# To support both python 2 and python 3
from __future__ import division, print_function, unicode_literals
from IPython.display import display,clear_output

import pandas as pd

import cx_Oracle
import os
import datetime

#parameters to connect to Oracle database (there is a Docker file at the repo 
#to create a database instance)
ip = 'database'
port = 1521
SID = 'sid'
user = 'user'
password = 'password'

dsn_tns = cx_Oracle.makedsn(ip, port, SID)
#encoding and access parameters depends on database installation
connection = cx_Oracle.connect(user, 
                               password, 
                               dsn_tns, 
                               encoding = "UTF-8", 
                               nencoding = "UTF-8")

**Mount google drive to access files**

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

**Information about the database**


*   Alarm table (SYSTEM.ALARMS): where alarms and events are logged. The most important columns are:
  * INTIME: datetime when event/alarm started;
  * OUTTIME: datetime when event/alarm ended;
  * MESSAGE: datetime description or message related to the alarm/event;
  * TAG: datetime device or localization related to the alarm/event;
  * AREA: lift station related to the alarm/event.
*   History tables: where timeseries are saved. In addition to the timestamp column, process data is found. The storage interval varies from one type of history to another.

**Some of the table and columns names were changed for security reasons**




In [0]:
#Important defines:
#total number of lift stations
NUMBER_OF_LIFT_STATIONS = 30 

#number of pumps per lift stations
PUMPS_PER_LIFT_STATION  = [2,2,3,3,3,1,2,3,2,2,3,3,2,2,2,
                           2,3,2,2,2,2,2,2,2,2,2,2,2,2,2]

#google drive folder where datasets are stored
DATASETS_DIR = r'/content/gdrive/My Drive/datasets/'                         


In [0]:
def create_csvfile_from_query(
    connection, query_format, filename_format, lif_station_index, pump_index): 
    """function to create csv file using query results:

    Args:
        connection (): connection to database.
        query_format (str): string format to query.
        filename_format (str): string format to filename.
        lif_station_index (int): lift station index.
        pump_index (int): pump index.                        

    Returns:
        Nothing
    """
    query        = query_format.format(lif_station_index, pump_index)
    outfilename  = filename_format.format(lif_station_index, pump_index)
    print(query)
    print(outfilename)    
    if 1:
        return
    bla = ble[334]
    #limit the chuncksize to avoid Memory Error
    reader       = pd.read_sql(query, connection, chunksize=50000)

    #read chuncks until the end
    for chunk in reader:
        temp_regs = chunk
        #append
        if os.path.isfile(outfilename):
            temp_regs.to_csv(outfilename,encoding='utf-8', mode='a', header=False)
        else:
            temp_regs.to_csv(outfilename,encoding='utf-8')



**Alarm table processing**

Alarm table is the biggest table with data from all the sewage lift stations

This table has a pattern determined by the SCADA System environment. This means that the table structure  is created by the product and the records creation are determined by the application the user created. 

In [0]:

#query format string to return alarms
query_format = """SELECT to_char(INTIME, 'yyyy-mm-dd HH24:MI:SS') as in_time,
                         to_char(OUTTIME, 'yyyy-mm-dd HH24:MI:SS') as out_time,
                         MESSAGE, TAG
                  FROM SYSTEM.ALARMS
                  WHERE AREA = 'area_{:0>2d}' 
                  ORDER BY INTIME"""

#filename format string to save alarms
filename_format = f'{DATASETS_DIR}alarms_area_'+'{:0>2d}.csv'

#reading alarm table from 
for idx in range(NUMBER_OF_LIFT_STATIONS):
    lift_station_idx = idx + 1

    #call function to create csv file
    create_csvfile_from_query(connection, 
                              query_format, 
                              filename_format, 
                              lift_station_idx,
                              0)
   
    temp_outfilename  = filename_format.format(lift_station_idx)
    final_outfilename = f'{DATASETS_DIR}alarms_area_{lift_station_idx:0>2d}w.csv'

    #creates the final version removing some inconsistency (duplicates)
    final_file = open(final_outfilename, 'w', encoding = "utf-8")
    last_row    = None #last row
    last_dh     = None #last dh
    cur_values  = [] #list of values to remove duplicates
    idxf = 0 #final file row count

    #the remove duplicates operation could be done with pandas operation
    #but this code avoid memory error

    #read the temp file removing duplicates
    with open(temp_outfilename, 'r', encoding = "utf-8") as temp_file:
        for line in temp_file:
            curdh   = line.split(',',2)[1] #datetime
            curline = line.split(',',1)[1] #entire line
            #check if timestamp has changed to initialize list of 
            #duplicated values
            if curdh != last_dh:
                cur_values = []

            #check if it is a duplicate before creating line
            if ((last_row == None) or (curline != last_row)) and \
                (curline not in cur_values):

                #if first line, writes the header
                if idxf == 0:
                    final_file.write(',%s'%(curline))
                else:
                    final_file.write('%d,%s'%(idxf - 1,curline))            
                idxf += 1
                cur_values.append(curline)
            last_row = curline
            last_dh = curdh
    
    #close files
    temp_file.close()
    final_file.close()




**History table related to power energy meter data (current, voltage and power)**

In [0]:
#query format string to return records
query_format = """SELECT to_char(record_timestamp, 
                                 'yyyy-mm-dd HH24:MI:SS') as timestamp, 
                      current_A,current_B,current_C,
                      voltage_RS,voltage_ST,voltage_RT,active_power,power_factor 
                FROM SYSTEM.INST_AREA_{0:0>2d}_MGE{1:0>2d}_{0}"""

#filename format string to save records into csv
filename_format = f'{DATASETS_DIR}pump_area_'+'{:0>2d}_{:0>2d}.csv'

for idx1 in range(NUMBER_OF_LIFT_STATIONS):
    lift_station_idx = idx1 + 1  
    for idx2 in range(PUMPS_PER_LIFT_STATION[idx]):      
        pump_idx = idx2 + 1
        #call function to create csv file
        create_csvfile_from_query(connection, 
                                  query_format, 
                                  filename_format, 
                                  lift_station_idx,
                                  pump_idx)




**History table related to instruments data (eg.: level and pumps current)**

In [0]:
#query format string to return records
query_format = """SELECT to_char(record_timestamp, 
                                'yyyy-mm-dd HH24:MI:SS') as timestamp, a.*
               FROM SYSTEM.TAB_INST_AREA_{0:0>2d} a"""

#filename format string to save records into csv
filename_format = f'{DATASETS_DIR}tab_inst_area_'+'{:0>2d}.csv'

#this history does not have pump
pump_idx = 0

for idx1 in range(NUMBER_OF_LIFT_STATIONS):
    lift_station_idx = idx1 + 1  
    #call function to create csv file
    create_csvfile_from_query(connection, 
                              query_format, 
                              filename_format, 
                              lift_station_idx,
                              pump_idx)

**History table related to system events**

In [0]:
#query format string to return records
query_format = """SELECT to_char(record_timestamp, 
                                 'yyyy-mm-dd HH24:MI:SS') as record_timestamp,
                         to_char(eventtime, 
                                 'yyyy-mm-dd HH24:MI:SS') as eventtime, 
                         event_message, event_comment
           from SYSTEM.SYS_EVENTS"""

#filename format string to save records into csv
filename_format = f'{DATASETS_DIR}tab_events.csv'

#this history does not have lift station or pump index
lift_station_idx = 0
pump_idx = 0

#call function to create csv file
create_csvfile_from_query(connection, 
                          query_format, 
                          filename_format, 
                          lift_station_idx,
                          pump_idx)

**History table related to totalizations per day**

In [0]:

#query format string to return records
query_format = """select to_char(record_timestamp, 
                                'yyyy-mm-dd HH24:MI:SS') as record_timestamp, 
                         a.*
               from SYSTEM.TAB_TOTPERDAY_AREA_{0:0>2d} a"""

#filename format string to save records into csv
filename_format = f'{DATASETS_DIR}tab_totperday_area_'+'{:0>2d}.csv'

#this history does not have pump
pump_idx = 0

for idx1 in range(NUMBER_OF_LIFT_STATIONS):
    lift_station_idx = idx1 + 1  
    #call function to create csv file
    create_csvfile_from_query(connection, 
                              query_format, 
                              filename_format, 
                              lift_station_idx,
                              pump_idx)

**History table related to the events written by the operators**

In [0]:
#query format string to return records
query_format = """select to_char(record_timestamp, 
                                 'yyyy-mm-dd HH24:MI:SS') as record_timestamp,
                         to_char(eventtime, 
                                 'yyyy-mm-dd HH24:MI:SS') as eventtime, a.*
           from SYSTEM.HIST_OPERATOR_EVENTS a"""

#filename format string to save records into csv
filename_format = f'{DATASETS_DIR}hist_operator_events.csv'

#this history does not have lift station or pump index
lift_station_idx = 0
pump_idx = 0

#call function to create csv file
create_csvfile_from_query(connection, 
                          query_format, 
                          filename_format, 
                          lift_station_idx,
                          pump_idx)

